<a href="https://colab.research.google.com/github/namwootree/Portfolio/blob/main/Alphaco_(Deep_Learning_Boot_Camp)/Long-Term%20Program/Text_Summarization/%5BMain%5D_Modeling_%26_Prediction_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install -q datasets transformers rouge-score nltk sentencepiece

     |████████████████████████████████| 362 kB 5.1 MB/s 
     |████████████████████████████████| 4.4 MB 63.8 MB/s 
     |████████████████████████████████| 1.2 MB 60.9 MB/s 
     |████████████████████████████████| 212 kB 62.0 MB/s 
     |████████████████████████████████| 1.1 MB 60.4 MB/s 
     |████████████████████████████████| 140 kB 59.8 MB/s 
     |████████████████████████████████| 86 kB 3.9 MB/s 
     |████████████████████████████████| 596 kB 52.1 MB/s 
     |████████████████████████████████| 127 kB 57.1 MB/s 
     |████████████████████████████████| 6.6 MB 53.3 MB/s 
     |████████████████████████████████| 94 kB 3.7 MB/s 
     |████████████████████████████████| 271 kB 67.0 MB/s 
     |████████████████████████████████| 144 kB 64.3 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which i

In [3]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer, AutoTokenizer, DataCollatorForSeq2Seq, EarlyStoppingCallback
from datasets import load_dataset, load_metric, Dataset
import pandas as pd
import numpy as np
import re
import nltk
import torch

In [4]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

### Load Model & Tokenizer & Metric

In [5]:
model_name = 'ainize/kobart-news'
metric_name = 'rouge'

In [6]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
metric = load_metric(metric_name)

Downloading:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


Downloading:   0%|          | 0.00/473M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/302 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/666k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

### Declear Functions

In [17]:
prefix = ""
# prefix = "summarize: "

max_input_length = 150
max_target_length = 150

def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["context"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["summary"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [18]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

### Load Data

In [19]:
train_path = '/content/drive/MyDrive/장기 프로젝트/문서 요약/augmented_data_set_141508.csv'
train_df = pd.read_csv(train_path, index_col=False)

In [20]:
split_ratio = 0.005

In [21]:
train_df.drop(['Unnamed: 0'], axis=1, inplace=True)
dataset = Dataset.from_pandas(train_df).train_test_split(split_ratio, seed=100)

In [22]:
dataset = dataset.map(preprocess_function, batched=True)

  0%|          | 0/141 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

### Training

In [23]:
batch_size = 1
num_train_epochs = 10
es = EarlyStoppingCallback(early_stopping_patience=3)
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model, padding=True)

In [24]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./log",
    num_train_epochs=num_train_epochs,
    evaluation_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    lr_scheduler_type='cosine',
    warmup_ratio=0.1,
    save_strategy='epoch',
    save_total_limit=3,
    fp16=True,
    predict_with_generate=True,
    # generation_max_length=,
    # generation_num_beams=,
    metric_for_best_model='eval_loss',
    load_best_model_at_end=True,
    gradient_accumulation_steps=16,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[es],
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Using cuda_amp half precision backend


In [ ]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: token_type_ids, summary, context. If token_type_ids, summary, context are not expected by `BartForConditionalGeneration.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 140800
  Num Epochs = 10
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 16
  Total optimization steps = 88000


Epoch,Training Loss,Validation Loss


In [ ]:
gdrive_path = '/content/drive/MyDrive/문서 요약'

In [ ]:
model.save_pretrained(f'{gdrive_path}/save')
tokenizer.save_pretrained(f'{gdrive_path}/save')

# Prediction

In [ ]:
prefix = ""

max_input_length = 200
max_target_length = 20

def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["context"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)
    
    return model_inputs

In [ ]:
gdrive_path = '/content/drive/MyDrive/문서 요약'

model = AutoModelForSeq2SeqLM.from_pretrained(f'{gdrive_path}/save')
tokenizer = AutoTokenizer.from_pretrained(f'{gdrive_path}/save')

In [ ]:
batch_size = 1
num_train_epochs = 10
es = EarlyStoppingCallback(early_stopping_patience=3)
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model, padding=True)

training_args = Seq2SeqTrainingArguments(
    output_dir="./log",
    num_train_epochs=num_train_epochs,
    evaluation_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    lr_scheduler_type='linear',
    warmup_ratio=0.1,
    save_strategy='epoch',
    save_total_limit=3,
    fp16=True,
    predict_with_generate=True,
    generation_max_length=30,
    generation_num_beams=5,
    metric_for_best_model='eval_loss',
    load_best_model_at_end=True,
    gradient_accumulation_steps=16,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    tokenizer=tokenizer,
    data_collator=data_collator
)

In [ ]:
test_path = '/content/drive/MyDrive/장기 프로젝트/문서 요약/test_df.csv'

test_df = pd.read_csv(test_path, index_col=False)
test_df.drop(['Unnamed: 0', 'id', 'title', 'region', 'agenda', 'total'], axis=1, inplace=True)
dataset = Dataset.from_pandas(test_df)

dataset = dataset.map(preprocess_function, batched=True)

In [ ]:
preds = trainer.predict(dataset)

The following columns in the test set don't have a corresponding argument in `MT5ForConditionalGeneration.forward` and have been ignored: summary, context. If summary, context are not expected by `MT5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 300
  Batch size = 1


In [ ]:
summary = []

for pred in preds.predictions:
    pred = tokenizer.decode(pred, skip_special_tokens=True)
    summary.append(pred)

In [ ]:
result = pd.read_csv('/content/sample_submission.csv')

result['summary'] = summary

result.to_csv('result_bart_r3f_14만개_3.csv', index=False)